In [1]:
import os, glob, re

import cv2

from deep_neural_network import face_classifier

In [2]:
CAPTURE_FILENAME_PREFIX = 'capture'
CAP_RE = re.compile('{0}([0-9]+).png'.format(CAPTURE_FILENAME_PREFIX))

def _get_cap_start_index():
    existing_cap_filenames = glob.glob('{0}*.png'.format(CAPTURE_FILENAME_PREFIX))
    matches = [CAP_RE.match(fn) for fn in existing_cap_filenames]
    indices = [int(m.group(1)) for m in matches if m is not None]
    if len(indices) > 0:
        return max(indices)
    else:
        return 0

CAP_INDEX = _get_cap_start_index() + 1
last_cap_filename = ''
print 'Capture file names start at {0}{1}.png'.format(CAPTURE_FILENAME_PREFIX, CAP_INDEX)

Capture file names start at capture8.png


In [3]:
def emotion_bar(image, index, probability, title):
    h = 30
    x = 250
    y = (index+1) * h
    w = int(probability * 100.0+0.5)
    cv2.putText(image, '{0}: {1:.1f}%'.format(title, probability*100.0), (0, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0,255,255))
    cv2.rectangle(image,(x,y-(h-2)),(x+w,y),(0,255,255),2)

In [4]:
cap = cv2.VideoCapture(0)

wrote_image_last_frame = False
while True:
    success, frame = cap.read()
    
    anno_frame = frame.copy()
    
    if not success:
        break
        
    Y_pred, face_boxes = face_classifier.detect_and_predict_face(frame)
    
    if Y_pred is not None:
        for (x,y,w,h) in face_boxes:
            cv2.rectangle(anno_frame,(x,y),(x+w,y+h),(255,0,0),2)
            
        for i, emotion in enumerate(face_classifier.EMOTIONS):
            emotion_bar(anno_frame, i, Y_pred[i], emotion)
        

    if wrote_image_last_frame:
        cv2.putText(anno_frame, 'Wrote {0}'.format(last_cap_filename), (0, 200),
                    cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255))
        
    cv2.imshow('Press ESC to exit ot SPACE to capture', anno_frame)
    kp = cv2.waitKey(5) 
    if kp == ord(' '):
        # Space
        if not wrote_image_last_frame:
            fn = '{0}{1}.png'.format(CAPTURE_FILENAME_PREFIX, CAP_INDEX)
            cv2.imwrite(fn, frame)
            last_cap_filename = fn
            CAP_INDEX += 1
        wrote_image_last_frame = True
    elif kp == 27:
        # ESC: exit
        wrote_image_last_frame = False
        break
    else:
        wrote_image_last_frame = False
        

        
cv2.destroyAllWindows()

cap.release()

/Users/Geoff/packages/Lasagne/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "
/Users/Geoff/packages/Lasagne/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "


AttributeError: 'NoneType' object has no attribute 'copy'